In [1]:
from models import *
from experiment import VAEXperiment
import torch.backends.cudnn as cudnn
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from torchvision import transforms
from utils.loaders import XrayImageDataset, CustomLoader, Img_To_1
import yaml
import argparse
import numpy as np
from PIL import Image

E:\py_env\main\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
with open(r"E:\Projects\vae-anomaly\configs\vanilla_vae.yaml", 'r') as file:
        try:
            config = yaml.safe_load(file)
        except yaml.YAMLError as exc:
            print(exc)

In [55]:

checkpoint = torch.load(r"E:\Projects\vae-anomaly\logs\VanillaVAE\fully_trained_20211022\checkpoints\epoch=11-step=384635.ckpt")


In [56]:
state_dict = {}
for key in checkpoint['state_dict']:
    state_dict[key.replace('model.','')] = checkpoint['state_dict'][key]

In [57]:
model = vae_models[config['model_params']['name']](**config['model_params'])

In [58]:
model.load_state_dict(state_dict)
model.eval()

VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (4): Seque

In [59]:

transform = transforms.Compose([transforms.ToTensor(), Img_To_1()])
dataset = XrayImageDataset(img_dir = r"D:\Documents\Anomaly\data",
                            transform=transform,
                            patch_size=config['exp_params']['patch_size'],
                            overlap=config['exp_params']['overlap'])

dataset_size = len(dataset)
split = int(np.floor(config['exp_params']['val_split'] * dataset_size))
train_size = dataset_size - split
test_size = split

training_set, validation_set = torch.utils.data.random_split(dataset, [train_size, test_size])

In [60]:
# Parameters
val_loader =  CustomLoader(validation_set, batch_size= 1,
                                                 drop_last=True)

In [122]:
for batch in val_loader:
    input_patch = batch.cpu().detach().numpy()
    input_patch = (input_patch*255).astype(np.int8)
    print(np.mean(input_patch[0, 0]))
    print(np.std(input_patch[0, 0]))
    img = Image.fromarray(input_patch[0, 0], 'L')
    img.save('input_patch.png')
    reconstruction = model.generate(batch)
    reconstruction_patch = reconstruction.cpu().detach().numpy()
    patch_image = (reconstruction_patch*255).astype(np.int8)
    print(np.mean(patch_image[0, 0]))
    print(np.std(patch_image[0, 0]))
    img = Image.fromarray(patch_image[0, 0], 'L')
    img.save('patch_image.png')
    break

AttributeError: 'NoneType' object has no attribute 'shape'